In [ ]:
# if need be, manually add the local project root to PYTHONPATH and move working directories

import os
import sys

project = '/' # change to local project root
sys.path.append(project)
os.chdir(project)

In [ ]:
# dependencies

import numpy as np
import pandas as pd

import xfx.lm.gibbs
import xfx.misc.plot

In [ ]:
# helper functions

def package_samples(samples, factor_names):

    rfx_samples, prec_samples, disp_samples = zip(*samples)
    rfx_samples = [np.array(samples_) for samples_ in zip(*rfx_samples)]
    prec_samples = np.array(prec_samples)
    disp_samples = np.array(disp_samples)
    return package_rfx_samples(rfx_samples, ['_const'] + factor_names), package_prec_samples(prec_samples, factor_names), pd.DataFrame(disp_samples).T.rename_axis(columns='iter')

def package_rfx_samples(rfx_samples, factor_names):

    dfs = []
    for samples_, factor_name in zip(rfx_samples, factor_names):
        df_ = pd.DataFrame(samples_.T)
        df_.index = df_.index.rename('level')
        df_.columns = df_.columns.rename('iter')
        df_['factor'] = factor_name
        dfs.append(df_)
    df = pd.concat(dfs).reset_index().set_index(['factor', 'level'])
    return df

def package_prec_samples(prec_samples, factor_names):

    df = pd.DataFrame(prec_samples.T, index=factor_names)
    df.index = df.index.rename('factor')
    df.columns = df.columns.rename('iter')
    return df

In [ ]:
# select predictors

factor_names = ['s', 'd', 'studage', 'lectage', 'service', 'dept']

In [ ]:
# configure algorithm (hyperprior on the random effect prior precision is Wishart(prior_pseudo_tau, prior_guess_tau / prior_pseudo_tau))

n_samples = 1000
n_warmup = 100
prior_pseudo_tau = np.ones(len(factor_names))
prior_guess_tau = np.ones(len(factor_names))
seed = 0

In [ ]:
# create inputs

insteval = pd.read_csv('demos/data/insteval.csv').loc[:, factor_names + ['y']]
n_obs = np.ones(len(insteval.y))
sum_obs = insteval.y
sumsq_obs = insteval.y ** 2
indices = insteval.reset_index().loc[:, factor_names].apply(lambda x: x.astype('category').cat.codes)
n_levels = indices.max(0).values + 1
rng = np.random.default_rng(seed)

In [ ]:
# sample

sampler = xfx.lm.gibbs.sample_posterior(sum_obs.values, sumsq_obs.values, n_obs, n_levels, indices.values, prior_pseudo_tau, prior_guess_tau, ome=rng)
samples = [next(sampler) for _ in range(n_samples + n_warmup)][n_warmup:]

In [ ]:
# reformat samples for plotting

rfx_samples, prec_samples, disp_samples = package_samples(samples, factor_names)

In [ ]:
# random effects samples

rfx_samples.iloc[:5, :5]

In [ ]:
# prior precision samples

prec_samples.iloc[:5, :5]

In [ ]:
# plot traces for 'studage' random effects, grid and color by level

xfx.misc.plot.plot_traces(rfx_samples.loc['studage'], 'iter', 'level', 'level')

In [ ]:
# plot marginals for 'lectage' random effects, grid and color by level

xfx.misc.plot.plot_marginals(rfx_samples.loc['lectage'], 'level', 'level')

In [ ]:
# plot ACFs for prior precisions, grid and color by level

xfx.misc.plot.plot_acf(prec_samples, 'factor', 'factor')